In [ ]:
import os
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import joblib

# Load model
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0)

def load_cropface_embeddings(folder):
    embeddings = []
    labels = []

    for filename in os.listdir(folder):
        if not filename.lower().endswith(('.jpg', '.png')):
            continue

        label = filename.split('_')[0]  # Lấy tên trước dấu _
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Không đọc được ảnh: {img_path}")
            continue

        img = cv2.resize(img, (112, 112))  # đảm bảo đúng input size
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Trích embedding từ ảnh đã crop
        embedding = app.models['recognition'].get_feat(img).flatten()
        embeddings.append(embedding)
        labels.append(label)

    return np.array(embeddings), np.array(labels)

# Bước 1: Load ảnh và trích embedding
X, y = load_cropface_embeddings("facecrop")
print(f"✅ Số ảnh: {len(X)}, nhãn: {set(y)}")

if len(X) < 2 or len(set(y)) < 2:
    raise ValueError("⚠️ Không đủ dữ liệu để train SVM. Cần ít nhất 2 ảnh từ 2 nhãn khác nhau.")

# Bước 2: Encode nhãn
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Bước 3: Train SVM
clf = SVC(kernel='linear', probability=True)
clf.fit(X, y_encoded)

# Bước 4: Lưu model
joblib.dump(clf, "svm_face_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("✅ Đã train & lưu model.")

# OMG IT WORKEDDDDDDDD AJNWDJNAWDJKNAWJDNKAWKJDAWNJDAWKJNDAWJKDJAKWDANWKJDJKNAWKDNJAWNJK


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
✅ Số ảnh: 49

/opt/conda/lib/python3.10/site-packages/sklearn/utils/multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_type = type_of_target(y, input_name="y")


✅ Đã train & lưu model.


In [5]:
import cv2
import numpy as np
import os
import joblib
from insightface.app import FaceAnalysis

# Load SVM model và LabelEncoder
clf = joblib.load("svm_face_model.pkl")
label_encoder = joblib.load("label_encoder.pkl")

# Load InsightFace
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0)

def predict_face(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Không thể đọc ảnh: {image_path}")

    img = cv2.resize(img, (112, 112))  # resize cho đúng kích thước
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Tạo giả định face object vì ảnh đã crop rồi
    face = type('Face', (object,), {})()
    face.kps = None  # Không có landmark
    embedding = app.models['recognition'].get_feat(img).flatten()

    # Predict
    pred = clf.predict([embedding])
    prob = clf.predict_proba([embedding])

    label = label_encoder.inverse_transform(pred)[0]
    confidence = prob[0][pred[0]]

    print(f"✅ Nhận diện: {label} (độ tự tin: {confidence:.2%})")

# Test một ảnh
predict_face("datasets/testcrop/test.png")  # Đổi path nếu cần



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
✅ Nhận diện:

In [15]:
import cv2
import numpy as np
import joblib
from insightface.app import FaceAnalysis

# Load SVM và label encoder
clf = joblib.load("svm_face_model.pkl")
label_encoder = joblib.load("label_encoder.pkl")

# Load InsightFace
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0)

# Khởi động webcam (nếu có webcam thật, hoặc đang dùng video mô phỏng)
cap = cv2.VideoCapture("output.avi")  # Đổi thành đường dẫn video nếu cần

frame_count = 0  # Đếm frame để debug nếu cần

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Chuyển sang RGB để insightface dùng
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect face
    faces = app.get(img_rgb)

    for face in faces:
        if face.embedding is None:
            continue  # Không nhận diện được

        embedding = face.embedding
        pred = clf.predict([embedding])[0]
        prob = clf.predict_proba([embedding])[0][pred]

        label = label_encoder.inverse_transform([pred])[0]

        # Vẽ bounding box + label
        x1, y1, x2, y2 = map(int, face.bbox)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} ({prob:.2%})", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # ✅ Thay vì imshow, ta lưu ảnh ra file (hoặc log ra console)
    output_path = f"output/frame_{frame_count}.jpg"
    cv2.imwrite(output_path, frame)
    print(f"[INFO] Frame {frame_count} saved to {output_path}")
    frame_count += 1

    # Chạy thử 50 frame rồi dừng (tránh chạy mãi trong Docker)
    if frame_count >= 50:
        break

cap.release()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
import os
import cv2

video_path = "output.avi"
print("📁 Current dir:", os.getcwd())
print("📂 Files in dir:", os.listdir("."))
print("🎥 Opening video:", video_path)

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("❌ Không mở được video. Có thể do:")
    print("  1. Video không tồn tại")
    print("  2. OpenCV không hỗ trợ codec (thiếu FFMPEG)")
    print("  3. File bị lỗi")
    exit()

📁 Current dir: /workspace/Supervise Learning
📂 Files in dir: ['.gradio', '.ipynb_checkpoints', 'CNN.ipynb', 'datasets', 'facecrop', 'FaceRegconition.ipynb', 'image', 'label_encoder.pkl', 'lfw_funneled', 'ML.ipynb', 'mnist_cnn_model.h5', 'PredictbyType.ipynb', 'PredictbyValue.ipynb', 'svm_face_model.pkl', 'Test Drawing Program.ipynb', 'yolov8n-face.pt']
🎥 Opening video: output.avi
❌ Không mở được video. Có thể do:
  1. Video không tồn tại
  2. OpenCV không hỗ trợ codec (thiếu FFMPEG)
  3. File bị lỗi


: 